# Gaussian Mixture Model
This notebook demonstrates unsupervised learning using the Gaussian Mixture Model.

In [ ]:
from sklearn.mixture import GaussianMixture
import numpy as np

# Sample data
X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])

# Define and fit the model
gmm = GaussianMixture(n_components=2, random_state=0).fit(X)
print(gmm.predict(X))